<a href="https://colab.research.google.com/github/AndyHerron/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [37]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-07 01:42:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-03-07 01:42:44 (1.22 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [38]:
# start a Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Challenge_Deliverable2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [39]:
# read in the data from S3 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [40]:
# Drop null values
dropna_df= df.dropna()

# Create the vine_table. DataFrame
from pyspark.sql.types import IntegerType
dropna_df = dropna_df.withColumn("star_rating", dropna_df["star_rating"].cast(IntegerType()))
vine_df = dropna_df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [41]:
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [42]:
# filter the dataframe to only include reviews with 20 or more votes
votes20_df = vine_df.filter(vine_df.total_votes >= '20')

In [43]:
# filter the votes20_df dataframe to only rows where helpful_votes/total_votes > 50%
helpful_votes_df = votes20_df.filter((votes20_df.helpful_votes / votes20_df.total_votes) >= '0.5')
helpful_votes_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|RXAHWIC1584UQ |5          |20           |23         |N   |Y                |
|RSBELAIC899DO |5          |46           |51         |N   |Y                |
|R2HCC4CJ59D225|4          |43           |43         |N   |Y                |
|RL9VF9WXHEHKR |5          |35           |36         |N   |Y                |
|R1ZINLWUOLTZ46|1          |39           |45         |N   |N                |
|R1CQKM1K1CHOHS|1          |21           |22         |N   |N                |
|R2DHAM7J1KMWN |5          |23           |24         |N   |N                |
|R26EZ2INBETPU2|1          |103          |128        |N   |N                |
|R265ZMDO2ISN1O|5          |20           |21         |N   |Y                |
|R3NZ7JTOCPYE1S|4          |32           |33         |N   |Y    

In [44]:
# filter the data to retrieve all the rows where a vine review is written
vine_reviews_df = helpful_votes_df.filter(helpful_votes_df.vine == "Y")
vine_reviews_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3QIQRG107AP19|5          |38           |41         |Y   |N                |
|R1ZNBHGCYE8VN1|5          |175          |181        |Y   |N                |
|R1BNZJ9IKGRBC2|4          |63           |70         |Y   |N                |
|R2IXTN4PBRHBO2|5          |19           |24         |Y   |N                |
|R32AOW3L09SYMQ|5          |29           |34         |Y   |N                |
|R34Z4VTLF7PC6X|4          |33           |36         |Y   |N                |
|R25403NE9JCRZZ|2          |36           |40         |Y   |N                |
|R2QT2G3YRGC0PS|5          |18           |20         |Y   |N                |
|RDUP15OURNUSA |5          |22           |23         |Y   |N                |
|R1EI4NZTG7G14F|5          |17           |23         |Y   |N    

In [45]:
# filter the data to retrieve all the rows that do not include a vine review
no_vine_df = helpful_votes_df.filter(helpful_votes_df.vine == "N")
no_vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|RXAHWIC1584UQ |5          |20           |23         |N   |Y                |
|RSBELAIC899DO |5          |46           |51         |N   |Y                |
|R2HCC4CJ59D225|4          |43           |43         |N   |Y                |
|RL9VF9WXHEHKR |5          |35           |36         |N   |Y                |
|R1ZINLWUOLTZ46|1          |39           |45         |N   |N                |
|R1CQKM1K1CHOHS|1          |21           |22         |N   |N                |
|R2DHAM7J1KMWN |5          |23           |24         |N   |N                |
|R26EZ2INBETPU2|1          |103          |128        |N   |N                |
|R265ZMDO2ISN1O|5          |20           |21         |N   |Y                |
|R3NZ7JTOCPYE1S|4          |32           |33         |N   |Y    

In [46]:
# determine the total number of reviews
vine_df.count()

1740974

In [47]:
# determine the number of 5 star reviews
vine_df.filter(vine_df.star_rating == "5").count()

1113482

In [50]:
# determine the percentage of 5 star reviews for vine_reviews_df
vine_5star= vine_reviews_df.filter(vine_reviews_df.star_rating == "5").count()
vine_percent = (vine_5star / vine_reviews_df.count()) * 100
# vine_reviews_df.count()
vine_5star
# vine_percent

163

In [52]:
# determine the percentage of 5 star reviews for no_vine_df
no_vine_5star= no_vine_df.filter(no_vine_df.star_rating == "5").count()
no_vine_percent = (no_vine_5star / no_vine_df.count()) * 100
# no_vine_percent
# no_vine_df.count()
no_vine_5star

14614

In [ ]:
# determine the percentage of 5 star reviews overall
all_5star= vine_df.filter(vine_df.star_rating == "5").count()
all_5star_percent = (all_5star / vine_df.count()) * 100
all_5star_percent

63.95741694017257

In [ ]:
# determine the percentage of 5 star reviews with 20 or more votes
filtered_5star= votes20_df.filter(votes20_df.star_rating == "5").count()
filtered_5star_percent = (filtered_5star / votes20_df.count()) * 100
filtered_5star_percent

42.84743266589255

In [ ]:
# determine the percentage of 5 star reviews within helpful_votes_df
helped_5star= helpful_votes_df.filter(helpful_votes_df.star_rating == "5").count()
helped_5star_percent = (helped_5star / helpful_votes_df.count()) * 100
helped_5star_percent

46.429132497564964